<a href="https://colab.research.google.com/github/Kieranosaurus/HYPER/blob/master/MXNet_PGGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install mxnet

import mxnet as mx
from mxnet import nd, gluon, init, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from IPython import display
import matplotlib.pyplot as plt
import time
import numpy as np

In [6]:
class Net(gluon.Block):

    def pixel_norm(x, epsilon=1e-8):
        return x * nd.rsqrt(nd.mean(nd.square(x), axis=1, keepdims=True) + epsilon)

    def apply_bias(x):
        b = nd.zeros(shape=[x.shape[1]])
        b = nd.cast(b, x.dtype)
        if len(x.shape) == 2:
            return x + b
        else:
            return x + nd.reshape(b, [1, -1, 1, 1])

    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            self.dense = nn.Dense(units=512, use_bias=False)
            self.lrelu = nn.LeakyReLU(alpha=0.2)
            self.conv1 = nn.Conv2D(channels=512, kernel_size=3, strides=(1,1), layout='NCHW', use_bias=False)

    def forward(self, x):
        # Layer 1
        x = pixel_norm(x)
        x = self.dense(x)
        x = nd.reshape(x, [-1, 512, 4, 4])
        x = pixel_norm(x)
        x = self.lrelu(apply_bias(x))
        x = self.lrelu(apply_bias(self.conv1(x)))
        # torgb? -> for intermediate output generation

        return x